# Анализ базы данных SQL

## Исследуем таблицы — выведем первые строки

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas




Выведем таблицу books

In [2]:
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем таблицу authors

In [3]:
query = '''SELECT * FROM  authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выведем таблицу publishers

In [4]:
query = '''SELECT * FROM  publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выведем таблицу ratings

In [5]:
query = '''SELECT * FROM  ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выведем таблицу reviews

In [6]:
query = '''SELECT * FROM  reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## SQL запросы 

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
           SELECT COUNT(book_id) 
           FROM  books 
           WHERE publication_date > '2000-01-01'
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Вышло 819 книг

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
query = '''
           SELECT b.book_id,
                  b.title,
                  COUNT(DISTINCT rv.review_id),
                  ROUND(AVG(rt.rating), 2)
           FROM  books AS b
           LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           GROUP BY b.book_id
           ORDER BY ROUND(AVG(rt.rating), 2) DESC, COUNT(DISTINCT rv.review_id) DESC
           
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count,round
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,642,The Big Bad Wolf (Alex Cross #9),2,5.00
4,967,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


Книга A Dirty Job (Grim Reaper #1) имеет набольшее количество обзоров при высшей оценке

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [9]:
query = '''
           SELECT p.publisher,
                  COUNT(b.book_id)
           FROM  books AS b
           LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
           WHERE b.num_pages > '50'
           GROUP BY p.publisher
           ORDER BY COUNT(b.book_id) DESC
           LIMIT 1
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


Издательство Penguin Books выпустило 42 книги

### Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками

In [10]:
query = '''
           WITH b_r AS (SELECT book_id,
                  COUNT(rating_id)
           FROM ratings
           GROUP BY book_id
           HAVING COUNT(rating_id) > '50')
           
           SELECT a.author,
                  ROUND(AVG(rt.rating), 2) AS rating
           FROM  books AS b
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           LEFT JOIN authors AS a ON b.author_id = a.author_id
           JOIN b_r ON b.book_id = b_r.book_id
           GROUP BY a.author
           ORDER BY rating DESC
           LIMIT 1
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.29


Автор J.K. Rowling/Mary GrandPré имеет самую высокую оценку 

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query = '''
           WITH u AS (SELECT username,
                  COUNT(rating_id)
           FROM ratings
           GROUP BY username
           HAVING COUNT(rating_id) > '48')
           
           SELECT COUNT(r.review_id) / COUNT(DISTINCT r.username) avg_review
           FROM reviews AS r
           JOIN u ON r.username = u.username
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review
0,24


Таких пользователей 24

## Вывод

- 819 книг вышло после 1 января 2000 года;
- Книга A Dirty Job (Grim Reaper #1) имеет набольшее количество обзоров при высшей оценке;
- Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц - 42;
- Автором с самой высокой средней оценкой книг с 50 и более оценками является J.K. Rowling/Mary GrandPré - 4.29;
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.